<a href="https://www.kaggle.com/code/lonnieqin/recommendation-system-with-tensorflow-recommenders?scriptVersionId=113996565" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Recommendation System with Tensorflow Recommenders
In this notebook, I will build a Recomender using Tensorflow Recommenders. I will demonstrate simple usage of Tensorflow Recommenders using MovieLens Dataset here. You can learn more about Tensorflow Recommenders in https://www.tensorflow.org/recommenders.
## Setup

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
tfx-bsl 1.5.0 requires absl-py<0.13,>=0.9, but you have absl-py 1.3.0 which is incompatible.
tfx-bsl 1.5.0 requires numpy<1.20,>=1.16, but you have numpy 1.20.3 which is incompatible.
tfx-bsl 1.5.0 requires pyarrow<6,>=1, but you have pyarrow 6.0.1 which is incompatible.
tensorflow-transform 1.5.0 requires absl-py<0.13,>=0.9, but you have absl-py 1.3.0 which is incompatible.
tensorflow-transform 1.5.0 requires numpy<1.20,>=1.16, but you have numpy 1.20.3 which is incompatible.
tensorflow-transform 1.5.0 requires pyarrow<6,>=1, but you have pyarrow 6.0.1 which is incompatible.
tensorflow-transform 1.5.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,<2.8,>=1.15.2, but you have tensorflow 2.11.0 which is incompati

In [2]:
import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
print(f"TF version: {tf.__version__}")
print(f"TF Recommender version: {tfrs.__version__}")

2022-12-16 14:09:56.406300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 14:09:56.590209: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-16 14:09:56.590248: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 14:09:58.099639: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

TF version: 2.11.0
TF Recommender version: v0.7.2


## Configuration

In [3]:
class Config: 
    
    embedding_dimension = 32
    
    model_path = "model.tf"
    
config = Config()

## Prepare the data

In [4]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

2022-12-16 14:10:03.237097: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteV6NM74/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


Let's take a look at the data.

In [5]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


2022-12-16 14:10:58.789112: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [7]:
for item in ratings.take(1):
    print(item)
for item in movies.take(1):
    print(item)

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>}
tf.Tensor(b'You So Crazy (1994)', shape=(), dtype=string)


2022-12-16 14:10:59.136253: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-12-16 14:10:59.194831: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Split dataset to training set and test set using Tensorflow API.

In [8]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [9]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## Build a two-tower retrieval model

## The query tower

In [10]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, config.embedding_dimension)
])

## The candidate tower

 

In [11]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, config.embedding_dimension)
])

## Loss

In [12]:
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
      candidates=movies.batch(128).map(movie_model)
    )
)

## MovieLensModel

In [13]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

## Model Training

In [14]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
cp = tf.keras.callbacks.ModelCheckpoint(config.model_path, save_best_only=True, save_weights_only=True)
es = tf.keras.callbacks.EarlyStopping(patience=4)
model.fit(cached_train, epochs=10, validation_data=cached_test, callbacks=[cp, es])
model.load_weights(config.model_path)

Epoch 1/10
10/10 [==============================] - 27s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_accuracy: 0.0103 - factorized_top_k/top_10_categorical_accuracy: 0.0215 - factorized_top_k/top_50_categorical_accuracy: 0.1013 - factorized_top_k/top_100_categorical_accuracy: 0.1759 - loss: 69888.5348 - regularization_loss: 0.0000e+00 - total_loss: 69888.5348 - val_factorized_top_k/top_1_categorical_accuracy: 0.0027 - val_factorized_top_k/top_5_categorical_accuracy: 0.0158 - val_factorized_top_k/top_10_categorical_accuracy: 0.0334 - val_factorized_top_k/top_50_categorical_accuracy: 0.1575 - val_factorized_top_k/top_100_categorical_accuracy: 0.2679 - val_loss: 28857.0820 - val_regularization_loss: 0.0000e+00 - val_total_loss: 28857.0820
Epoch 2/10
10/10 [==============================] - 19s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0032 - factorized_top_k/top_5_categorical_accuracy: 0.0215 - factorized_top_k/top_10_ca

## Making Recommendations

In [15]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=12)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)
# Get recommendations.
ids = ["42", "43", "44"]
_, titles = index(tf.constant(ids))
for i in range(len(ids)):
    print(f"Recommendations for user id {ids[i]}: {titles[i].numpy()}")

Recommendations for user id 42: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)'
 b"Kid in King Arthur's Court, A (1995)" b'Michael (1996)'
 b"Preacher's Wife, The (1996)" b'101 Dalmatians (1996)'
 b'Rent-a-Kid (1995)' b'Angels in the Outfield (1994)' b'Rudy (1993)'
 b'Houseguest (1994)' b'Grumpier Old Men (1995)'
 b'Up Close and Personal (1996)']
Recommendations for user id 43: [b'Bed of Roses (1996)' b'One Fine Day (1996)'
 b'How to Make an American Quilt (1995)' b'French Kiss (1995)'
 b'Up Close and Personal (1996)' b'While You Were Sleeping (1995)'
 b'Now and Then (1995)' b'Cool Runnings (1993)' b'Sabrina (1995)'
 b'With Honors (1994)' b'Powder (1995)' b'Mr. Wrong (1996)']
Recommendations for user id 44: [b'Star Trek V: The Final Frontier (1989)'
 b'Star Trek IV: The Voyage Home (1986)' b'Forbidden Planet (1956)'
 b'Raiders of the Lost Ark (1981)' b'Wizard of Oz, The (1939)'
 b'Aristocats, The (1970)' b'Aladdin (1992)' b'Die Hard (1988)'
 b'Day the E

Making recommendations is really fast. It can make about 30000 recommendations per second by using only CPU.

In [16]:
%%time
import time
y_preds = None
user_ids = tf.data.Dataset.from_tensor_slices((unique_user_ids)).batch(256)
begin = time.time()
for item in user_ids:
    _, titles = index(item)
    if y_preds is None:
        y_preds = titles
    else:
        y_preds = tf.concat([y_preds, titles], axis=0)
elapsed = time.time() - begin
print(f"Inference speed:{int(y_preds.shape[0] / elapsed)} samples/s")
print(y_preds[0:10])

Inference speed:25410 samples/s
tf.Tensor(
[[b'Four Rooms (1995)' b'Swimming with Sharks (1995)' b'Supercop (1992)'
  b'Bottle Rocket (1996)' b'Blue in the Face (1995)' b'Akira (1988)'
  b'Grand Day Out, A (1992)' b'Chungking Express (1994)'
  b'Close Shave, A (1995)' b'Swingers (1996)'
  b'Three Colors: White (1994)' b'Clerks (1994)']
 [b'Wonderful, Horrible Life of Leni Riefenstahl, The (1993)'
  b'Paths of Glory (1957)' b'Sunset Blvd. (1950)'
  b'Third Man, The (1949)' b'East of Eden (1955)'
  b'Thirty-Two Short Films About Glenn Gould (1993)'
  b'Some Like It Hot (1959)' b'Raging Bull (1980)'
  b'All About Eve (1950)' b'Thin Man, The (1934)' b'M (1931)'
  b'8 1/2 (1963)']
 [b'Truman Show, The (1998)' b'Apostle, The (1997)'
  b'Great Expectations (1998)' b'Washington Square (1997)'
  b'Mrs. Dalloway (1997)' b'Blues Brothers 2000 (1998)'
  b'Deep Rising (1998)' b'Welcome To Sarajevo (1997)'
  b'Wag the Dog (1997)' b'Hard Rain (1998)' b'Amistad (1997)'
  b'Palmetto (1998)']
 [b'First 